In [60]:
# Import dependencies
import subprocess
import json
from dotenv import load_dotenv

# Load/ set environment variables and mnemonic key
import os
load_dotenv()
mnemonickey=os.getenv("MNEMONICKEY")
#print(mnemonickey) only for testing

# Import constants.py and necessary functions from bit and web3
from constants import *
from bit import wif_to_key
from web3 import Web3
from bit import *
from web3.middleware import geth_poa_middleware
from eth_account import Account
from pathlib import Path
from getpass import getpass
from bit.network import NetworkAPI

In [61]:
# local host provider set up
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545")) 
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [62]:
# Create a function called `derive_wallets' using the mnemonic key
def derive_wallets(mnemonickey, coin, numderive):
    #command = f'C:/Users/User/Documents/Renu/Homework19/wallet/hd-wallet-derive/php derive -g --mnemonic="mnemonickey"  --coin={coin} --numderive={numderive} --cols=path,address,privkey,pubkey --format=json'
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonickey}" --numderive="{numderive}" --coin="{coin}" --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    #print(output)
    p_status = p.wait()
    keys = json.loads(output)
    return keys  

In [66]:
# Create a dictionary object to store output from `derive_wallets`
coins = {"eth", "btc-test"}
coins

{'btc-test', 'eth'}

In [67]:
# Creating the dictionary
coin_keys = {}
for coin in coins:
    coin_keys[coin]= derive_wallets(mnemonickey, coin, 2)    

In [68]:
#json in readable format- need the private key
print(json.dumps(coin_keys, indent=2, sort_keys=True))

{
  "btc-test": [
    {
      "address": "mqE7qKYrpthCwKuQhgRjC6BejVzic69MEg",
      "index": 0,
      "path": "m/44'/1'/0'/0/0",
      "privkey": "cQ56Nxc9kfrXpuQCiLRLELvJrwPF7MiesK5uS6wfXAiMMLJEUTrE",
      "pubkey": "0240cfb30a9ebfde18aa1a588085c51728195d9cd8c69f4744100a60cb863903fa",
      "pubkeyhash": "6a8226392ccb95c3e8960a73b1e913e867016324",
      "xprv": "tprv8kJvZcLqmz7j52PvtUdCg6rbd5niZBPrY84wWTE9y1wYNA88skiZ6JKFKyuJArzNDcHH1xa9NpDauVcwMFjexgK6P249dVAWGfL3QtJCYwb",
      "xpub": "tpubDGzxi2P5vMoPxVRin8Ho5WWiC7JeiWam7RfinyGTPHjwCeNuW9Y9Gnw7W6n2FVpmvtre3pjAWzh1wv1YgfwvHxXT7b4RG6X5vcVZjf1ZFUD"
    },
    {
      "address": "mv9iLPQ4q2kxzTj2xFuFzzcfdZmN3yCCjA",
      "index": 1,
      "path": "m/44'/1'/0'/0/1",
      "privkey": "cMytcYT5tvgPK6JNDgrNJ4dLQc4CestV87viphF52PRrG5wsqbVL",
      "pubkey": "025eb3f50260d69899bf4031b5608cd4a96ba286c739ae8b85eb7a1525876ab0e2",
      "pubkeyhash": "a0857f9f4d506bc4cfe12e708303a8e7dade5050",
      "xprv": "tprv8kJvZcLqmz7j93U259KTFXSZA7UxF

In [69]:
# function to get private key
def priv_key_to_account(coin, private_key):
    if coin == ETH:
        return Account.privateKeyToAccount(private_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(private_key)    

In [70]:
# get the private keys generated from the derive wallet function - first one
eth_private_key = coin_keys["eth"][0]['privkey']
btctest_private_key = coin_keys['btc-test'][0]['privkey']
print(eth_private_key)
print(btctest_private_key)

0x37a9a47c4cfe509c44bd9b66ee8cabc940a737965ab76f32810dcee932c5cb57
cQ56Nxc9kfrXpuQCiLRLELvJrwPF7MiesK5uS6wfXAiMMLJEUTrE


In [71]:
# Added this extra note- Comparison of keys to previous version of this code where the same private keys were obtained from the environment varaible file.
eth_private_key = os.getenv("ETH_PRIVATEKEY")
btctest_private_key = os.getenv("BTCTEST_PRIVATEKEY")
print (eth_private_key)
print(btctest_private_key)
#both sets of private keys are the same- match!!!

0x37a9a47c4cfe509c44bd9b66ee8cabc940a737965ab76f32810dcee932c5cb57
cQ56Nxc9kfrXpuQCiLRLELvJrwPF7MiesK5uS6wfXAiMMLJEUTrE


In [72]:
# get private key for ETH
ETH_key = priv_key_to_account(ETH, eth_private_key)
print(ETH_key)

In [73]:
# get private key for BTCTEST
BTCTEST_key = priv_key_to_account(BTCTEST,btctest_private_key)
print(BTCTEST_key)

<PrivateKeyTestnet: mqE7qKYrpthCwKuQhgRjC6BejVzic69MEg>


In [75]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_raw_tx(coin, account, recipient, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
        {"from": account.address, "to": recipient, "value": amount}
        )
        return {
        "from": account.address,
        "to": recipient,
        "value": amount,
        "gasPrice": w3.eth.gasPrice,
        "gas": gasEstimate,
        "nonce": w3.eth.getTransactionCount(account.address),
    }
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [76]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, recipient, amount):
    tx = create_raw_tx(coin, account, recipient, amount)
    signed_tx = account.sign_transaction(tx)
    if coin == ETH:
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        return result.hex()
    elif coin == BTCTEST:
        return NetworkAPI.broadcast_tx_testnet(signed_tx)

In [77]:
# print metamask account 
print(ETH_key.address)

0x08742B7fd52FfF993c7220a5a074AC4cD240790b


In [44]:
#print metamask balance before transaction
w3.eth.get_balance(ETH_key.address)

99994311959999994120

In [45]:
#send ETH transaction from metmask account one to metamask account 2- refer Ganache screen shot
send_tx(ETH, ETH_key, "0xF5755e05aE47B8560414e9fd6fFE6Aa2bd02c36e", 1960)

'0x120477454c55cb98953957df5093a11f8bd472cf1ab230355bb44ccf6d8a7d66'

In [46]:
#print metamask balance in wei after transaction- metamask account balance screen shot
w3.eth.get_balance(ETH_key.address)

99993891959999992160

In [52]:
#send BTCTEST transaction to another account (Chirath) and check in Blockcypher- max 8 decimals
send_tx(BTCTEST, BTCTEST_key, "mq5nwjbW5GqLMeEnvpUaBLkwi7SHKMe6WU", .00000015)

In [53]:
#get transaction details
BTCTEST_key.get_transactions()

['453ba2baf2f510c9bdd53164c252387d3cd4204b11017ae5921a00afaf4b9ede',
 '7c79a1c6d7e0a3682205d53af6e08308eabf31a7cdf0da1691e4c82704e0bb03',
 'b9cb1f05ed4e5bfa85890a535e071e482f9351faf219a69e73d1e9437ca16351',
 '94d924c3c148cffe077c8d0a05151a505ac5e02241e79c75c9a6a5e06cd5ae93',
 '506cada442bb5eb180cb79e69fe989f421a9e309331df6a105ee64592e062cf2',
 '7540cf139f92d601cf9edd887a1c5be72ec681d93aa6bf22861cfab5177070d3',
 '73e30130564b8be7328b0d35fc48c21a3cbb2b27cfa82fc4e1ad014d2d6d39b0',
 'a3b412affa2ab4ed5789c35a6835a7f1925efcc2e64e6ad785799409cafe7a93']

In [54]:
#balance after successful transaction- refer screen shot from Blockcypher
BTCTEST_key.balance_as('satoshi')

'0'

In [199]:
#The End